In [521]:
import numpy as np
import matplotlib.pyplot as plt
import wikipediaapi
import scipy.sparse as sps
import scipy.sparse.linalg as spl
import seaborn
import csv

alpha=0.85

In [585]:
#Power iteration avec la somme
def power_iteration_somme(dict1,alpha,epsilon):
    nb_sommet=len(dict1)
    y=(np.ones(nb_sommet))*(1-alpha)/nb_sommet
    x0=np.zeros(nb_sommet)
    x1=np.ones(nb_sommet)
    iteration=0
    while(np.linalg.norm(x1-x0)>epsilon): 
        iteration=iteration+1
        x0=np.copy(x1)
        for cles in dict1:
            somme=0
            for cles2 in dict1: 
                if cles in dict1[cles2]:
                    somme=somme+(x0[cles2]/len(dict1[cles2]))
            x1[cles]=alpha*somme+y[cles]
    return x1,iteration

dict2={0:{0,1,2}, 1:{0,1}, 2:{2}}
print(power_iteration_somme(dict2,0.85,1e-10))

#Valeur exacte
e=np.ones(len(dict2))
P=np.array([[1/3,1/3,1/3],[1/2,1/2,0],[0,0,1]])
A=np.transpose((np.identity(3)-alpha*P))
B=((1-alpha)/len(dict2))*np.transpose(e)
pi=np.linalg.solve(A,B)
print("valeur exacte",pi)


(array([0.17142857, 0.17142857, 0.65714286]), 136)
valeur exacte [0.17142857 0.17142857 0.65714286]


In [586]:
def power_iteration_somme_perso(dict1,alpha,epsilon,liste):
    nb_sommet=len(dict1)

    #creation y
    y=np.zeros(nb_sommet)
    for i in liste:
        y[i]=1
    x1=y/len(liste)
    y=(1-alpha)*y/len(liste)
    x0=np.zeros(nb_sommet)
    iteration=0
    while(np.linalg.norm(x1-x0)>epsilon): 
        iteration=iteration+1
        x0=np.copy(x1)
        for cles in dict1:
            somme=0
            for cles2 in dict1: 
                if cles in dict1[cles2]:
                    somme=somme+(x0[cles2]/len(dict1[cles2]))
            x1[cles]=(alpha*somme)+y[cles]
    return x1,iteration
dict2={0:{0,1,2}, 1:{0,1}, 2:{2}}
liste=[1]
print(power_iteration_somme_perso(dict2,0.85,1e-10,liste))

#Valeur exacte
e=[0,1,0]
P=np.array([[1/3,1/3,1/3],[1/2,1/2,0],[0,0,1]])
A=np.transpose((np.identity(3)-alpha*P))
B=(1-alpha)*np.transpose(e)
pi=np.linalg.solve(A,B)
print("valeur exacte",pi)

(array([0.21857143, 0.36857143, 0.41285714]), 64)
valeur exacte [0.21857143 0.36857143 0.41285714]


In [491]:
#Power iteration avec matrice
def creation_p(dict2): # conversion de notre dictionnaire à la matrice de transition
    nb_sommet=len(dict2)
    P=np.zeros((nb_sommet,nb_sommet))
    for cle in dict2:
        for sommet in dict2[cle]:
            P[cle,sommet]=1/len(dict2[cle])
    return P

def power_iteration_matrice(dict2,pi,alpha): # calcul de la valeur du page rank de manière matricielle
    nb_sommet=len(dict2)
    P=creation_p(dict2)
    constante=((1-alpha)/nb_sommet)*np.ones(nb_sommet)
    return alpha*np.dot(pi,P)+constante

#Test:
dict2={0:{1,2}, 1:{2}, 2:{0}}
tab=np.ones(len(dict2))/len(dict2)
pi_0=np.ones((1,len(dict2)))/len(dict2)
alpha=0.85

#Erreur
page_rank_2=power_iteration_matrice(dict2,pi_0,alpha)
iteration2=1

epsilon=0.00000000000001
while(np.linalg.norm(page_rank_2-pi_0)>epsilon): 
    iteration2=iteration2+1
    pi_0=page_rank_2
    page_rank_2=power_iteration_matrice(dict2,pi_0,alpha)

print(page_rank_2)
print(iteration2)


[[0.38778971 0.21481063 0.39739966]]
62


In [492]:
def power_iteration_matrice_erreur(dict2,alpha,epsilon): # calcul de la valeur du page rank de manière matricielle avec erreur epsilon
    nb_sommet=len(dict2)
    P=creation_p(dict2)
    e=np.array([(1-alpha)/nb_sommet]*nb_sommet)
    pi_avant=np.array(e)
    pi=[0]*nb_sommet
    iteration=0
    while(np.linalg.norm(pi-pi_avant)>epsilon):
        pi_avant=pi
        pi=alpha*np.dot(pi_avant,P)+e
        iteration=iteration+1
    return pi,iteration

#Test:
dict2={0:{0,1,2}, 1:{1,2}, 2:{0,2}}
print(power_iteration_matrice_erreur(dict2,alpha,1e-10))

(array([0.32421561, 0.24671494, 0.42906946]), 128)


In [526]:
"""#Power iteration avec sparse
def dict_sparse(dict2):
    data=[]
    row_ind=[]
    col_ind=[]
    for cle in dict2:
        for sommet in dict2[cle]:
            data.append(1/len(dict2[cle]))
            row_ind.append(cle)
            col_ind.append(sommet)
    return scipy.sparse.csr_matrix((data,(row_ind,col_ind)),shape=(len(dict2),len(dict2)))

def dict_sparse2(dict2):
    data=[]
    row_ind=[]
    col_ind=[]
    for cle in dict2:
        for sommet in dict2[cle]:
            row_ind.append(cle)
            col_ind.append(sommet)
    return row_ind,col_ind

P_sparse=dict_sparse(dict2)
print(type(P_sparse))
print(P_sparse)

def power_iteration_sparse(dict2,pi,alpha):
    nb_sommet=len(dict2)
    P=dict_sparse(dict2)
    constante=((1-alpha)/nb_sommet)*np.ones(nb_sommet)
    P_t=P.transpose()
    pi_t=pi.transpose()
    return (alpha*P_t.dot(pi_t)).transpose()+constante

def power_iteration_sparse2(row,col,pi,alpha,n):
    valeurs=[1]*len(row)
    P=scipy.sparse.csr_matrix((valeurs,(row,col)))
    constante=((1-alpha)/n)*np.ones(n)
    P_t=P.transpose()
    pi_t=pi.transpose()
    return (alpha*P_t.dot(pi_t)).transpose()+constante


#Test:
dict2={0:{1,2}, 1:{2}, 2:{0}}
pi_0=np.ones((1,len(dict2)))/len(dict2)
alpha=0.85
row_ind,col_ind=dict_sparse2(dict2)

#Erreur
page_rank_2=power_iteration_sparse2(row_ind,col_ind,pi_0,alpha,3)
iteration2=1

epsilon=0.00000000000001
while(np.linalg.norm(page_rank_2-pi_0)>epsilon): 
    iteration2=iteration2+1
    pi_0=page_rank_2
    page_rank_2=power_iteration_sparse2(row_ind,col_ind,pi_0,alpha,3)

"""

SyntaxError: invalid syntax (3360201666.py, line 60)

In [531]:
def reader_lists(fileName):
	with open(fileName, 'r') as csvfile:
		rows = []
		cols = []
		edgelist = csv.reader(csvfile, delimiter=' ')
		for line in edgelist:
			rows.append( int(line[0]) )
			cols.append( int(line[1]) )

		return rows, cols
	

def pagerank_sparse(rows, cols, alpha, epsilon):
	size = max(max(rows), max(cols)) + 1
	A = sps.csr_matrix(([1]*len(rows), (rows, cols)), shape=(size,size))
	A = A + sps.eye(size)
	D = A.dot([1]*size)
	Dinv = sps.diags([np.reciprocal(D)], [0])
	P = Dinv.dot(A)
	
	e=np.array([(1-alpha)/size]*size)
	pi_avant=np.array(e)
	pi=[0]*size
	Pt=P.transpose()

	while(np.linalg.norm(pi-pi_avant)>epsilon):
		pi_avant=pi
		pi=alpha*(Pt.dot(pi_avant))+e
	return pi


dict2={0:{1,2}, 1:{0}, 2:{}} 
rows,cols=dict_sparse2(dict2)
pi_0=np.ones((1,max(max(rows), max(cols)) + 1))/(max(max(rows), max(cols)) + 1)
page_rank=pagerank_sparse(rows,cols,0.85,1e-10)
print(page_rank)



[0.17142857 0.17142857 0.65714286]


In [532]:
def pagerank_sparse_perso(rows, cols, alpha, epsilon,liste):
	size = max(max(rows), max(cols)) + 1
	A = sps.csr_matrix(([1]*len(rows), (rows, cols)), shape=(size,size))
	A = A + sps.eye(size)
	D = A.dot([1]*size)
	Dinv = sps.diags([np.reciprocal(D)], [0])
	P = Dinv.dot(A)

	#creation de y
	y=np.zeros(size)
	for i in liste:
		y[i]=1
	pi=np.array(y)
	y=(1-alpha)*y/len(liste)
	pi_avant=[0]*size
	Pt=P.transpose()
	iteration=0
	while(np.linalg.norm(pi-pi_avant)>epsilon):
		pi_avant=pi
		pi=alpha*(Pt.dot(pi_avant))+y
		iteration=iteration+1
	return pi,iteration

rows=[0,0,1]
cols=[1,2,0]
liste=[1]
print(pagerank_sparse_perso(rows,cols,0.85,1e-10,liste))

(array([0.21857143, 0.36857143, 0.41285714]), 64)


In [593]:
#Fonction qui calcule la valeur exacte du page rank en prenant en entrée tableau de lignes et de colonnes 
def exact_pr(rows, cols, alpha):
	size = max(max(rows), max(cols)) + 1
	A = sps.csr_matrix(([1]*len(rows), (rows, cols)), shape=(size,size))
	A = A + sps.eye(size)
	D = A.dot([1]*size)
	Dinv = sps.diags([np.reciprocal(D)], [0])
	P = Dinv.dot( A ).transpose()
	e = [1/size]*size
	M = sps.eye(size) - alpha*P
	pr_exact = spl.spsolve(M, e)
	return pr_exact

def dico_construction(rows,cols):
	size = len(rows)
	dico = dict()
	for i in range(size):
		if rows[i] not in dico: dico[rows[i]] = set([rows[i]])	
		if cols[i] not in dico: dico[cols[i]] = set([cols[i]])
		dico[rows[i]].add( cols[i] )
	return dico
	
#Forward push avec dictionnaire en entrée
def forward_push(dico, alpha, epsilon):
	size = len(dico)
	r = np.array([(1-alpha)/size]*size)
	p = np.zeros(size)
	counter = 0
	while max( r ) > epsilon : 
		u = np.argmax( r )
		x = r[u]
		p[u] += x
		r[u] -= x
		
		voisins = dico[u]
		num_voisins = len(voisins)
		for i in voisins:
			r[i] += alpha*x/num_voisins		
	return p

rows=[0,0,1]
cols=[1,2,0]
dico=dico_construction(rows,cols)
print(dico)
print(forward_push(dico,0.85,1e-10))

{0: {0, 1, 2}, 1: {0, 1}, 2: {2}}
[0.17142857 0.17142857 0.65714286]


In [596]:
#Forward push avec dictionnaire en entrée page rank perso
def forward_push_perso(dico, alpha, epsilon,liste):
	size = len(dico)
	#creation y
	y=np.zeros(size)
	for i in liste:
		y[i]=1
	r = y*(1-alpha)/len(liste) # ici r c'est y*(1-alpha)
	p = np.zeros(size)
	counter = 0
	while max( r ) > epsilon : 
		u = np.argmax( r )
		x = r[u]
		p[u] += x
		r[u] -= x
		
		voisins = dico[u]
		num_voisins = len(voisins)
		for i in voisins:
			r[i] += alpha*x/num_voisins		
	return p

rows=[0,0,1]
cols=[1,2,0]
dico=dico_construction(rows,cols)
liste=[1]
print(dico)
print(forward_push_perso(dico,0.85,1e-10,liste))

{0: {0, 1, 2}, 1: {0, 1}, 2: {2}}
[0.21857143 0.36857143 0.41285714]


In [599]:
def pr_power_iteration_somme(dico,alpha,epsilon,perso,liste=[]):
    nb_sommet=len(dico)
    if perso:
        y=np.zeros(nb_sommet)
        for i in liste:
            y[i]=1
        x1=y/len(liste)
        y=(1-alpha)*y/len(liste)
    else:
        y=(np.ones(nb_sommet))*(1-alpha)/nb_sommet
        x1=np.ones(nb_sommet)
    x0=np.zeros(nb_sommet)
    iteration=0
    while(np.linalg.norm(x1-x0)>epsilon):
        x0=np.copy(x1)
        for cles in dico:
            somme=0
            for cles2 in dico:
                if cles in dico[cles2]:
                    somme=somme+(x0[cles2]/len(dico[cles2]))
            x1[cles]=(alpha*somme)+y[cles]
    return x1

dico={0:{0,1,2}, 1:{0,1}, 2:{2}}
liste=[1]

print("perso",pr_power_iteration_somme(dico,0.85,1e-10,True,liste))
print("normale",pr_power_iteration_somme(dico,0.85,1e-10,False))

perso [0.21857143 0.36857143 0.41285714]
normale [0.17142857 0.17142857 0.65714286]


In [592]:
def pr_power_iteration_sparse(rows,cols, alpha, epsilon,perso,liste=[]):

    size=max(max(rows),max(cols))+1
    A=sps.csr_matrix(([1]*len(rows),(rows,cols)),shape=(size,size))
    A=A+sps.eye(size)
    D=A.dot([1]*size)
    Dinv=sps.diags([np.reciprocal(D)],[0])
    P=Dinv.dot(A)


    #creation de y  
    if perso:
        y=np.zeros(size)
        for i in liste:
            y[i]=1
        pi=np.array(y)
        y=(1-alpha)*y/len(liste) 
        pi_avant=[0]*size 
    
    else:
        y=np.array([(1-alpha)/size]*size)
        pi=[0]*size
        pi_avant=np.array(y)

    #Calcul du P transpose
    Pt=P.transpose()


    #Calcul du page rank
    while(np.linalg.norm(pi-pi_avant)>epsilon):
        pi_avant=pi
        pi=alpha*(Pt.dot(pi_avant))+y
    
    return pi

rows=[0,0,1]
cols=[1,2,0]
liste=[1]

print("perso",pr_power_iteration_sparse(rows,cols,0.85,1e-10,True,liste))
print("normale",pr_power_iteration_sparse(rows,cols,0.85,1e-10,False))

perso [0.21857143 0.36857143 0.41285714]
normale [0.17142857 0.17142857 0.65714286]


In [600]:
def pr_forward_push(dico,alpha,epsilon,perso,liste=[]):
    size=len(dico)

    #creation de y
    if perso:
        y=np.zeros(size)
        for i in liste:
            y[i]=1
        r=y*(1-alpha)/len(liste)

    else:
        r=np.array([(1-alpha)/size]*size)


    #calcul du page rank
    p=np.zeros(size)


    while max(r)>epsilon:
        u=np.argmax(r)
        x=r[u]
        p[u]+=x
        r[u]-=x

        voisins=dico[u]
        num_voisins=len(voisins)

        for i in voisins:
            r[i]+=alpha*x/num_voisins

    return p

dico={0:{0,1,2}, 1:{0,1}, 2:{2}}
liste=[1]
print("perso",pr_forward_push(dico,0.85,1e-10,True,liste))
print("normale",pr_forward_push(dico,0.85,1e-10,False))

perso [0.21857143 0.36857143 0.41285714]
normale [0.17142857 0.17142857 0.65714286]
